In [1]:


from PIL import Image
import numpy as np
import os
from keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(12)


2024-04-21 14:48:53.803208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


LOAD IN ALL TRAINING IMAGES TO NP ARRAY

In [87]:
# load images into np array
n_images = 0
for dir_name in os.listdir("images/train"):
    if dir_name != "uncertain_train" and os.path.isdir(f"images/train/{dir_name}"):
        for filename in os.listdir(f"images/train/{dir_name}"):
            n_images += 1
images = np.empty((n_images, 224, 224, 3))
for dir_name in os.listdir("images/train"):
    index = 0
    print(dir_name)
    if dir_name != "uncertain_train" and os.path.isdir(f"images/train/{dir_name}"):
        for filename in os.listdir(f"images/train/{dir_name}"):
            img = kimage.load_img(f"images/train/{dir_name}/{filename}", target_size=(224, 224))
            img_array = kimage.img_to_array(img)
            img_array = preprocess_input(img_array)
            index += 1
            images[index] = img_array

Hernia
Mass
uncertain
.DS_Store
Cardiomegaly
Atelectasis
Effusion
Pneumothorax
No Finding
Nodule
Edema
Pleural_Thickening
Pneumonia
Emphysema
Infiltration
Consolidation
Fibrosis


INITIALIZE RESNET50 MODEL WITH ADAM OPTIMIZER, CATEGORICAL CROSS-ENTROPY LOSS

In [5]:
# assemble ResNet50 model, using pretrained weights on ImageNet
base_model = ResNet50(include_top=False, 
                            classes=15,
                            input_shape=(224,224,3),
                            weights=None)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'images/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        verbose=1)

2024-04-21 01:57:44.035056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 5859 images belonging to 15 classes.
Epoch 1/5
184/184 [==============================] - 958s 5s/step - loss: 1.6477 - accuracy: 0.6747
Epoch 2/5
184/184 [==============================] - 948s 5s/step - loss: 1.3618 - accuracy: 0.6919
Epoch 3/5
184/184 [==============================] - 953s 5s/step - loss: 1.3444 - accuracy: 0.6894
Epoch 4/5
184/184 [==============================] - 976s 5s/step - loss: 1.3417 - accuracy: 0.6914
Epoch 5/5
184/184 [==============================] - 975s 5s/step - loss: 1.3200 - accuracy: 0.6921


In [2]:
# model checkpoints
checkpoint_path = "images/train"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


# assemble ResNet50 model, using pretrained weights on ImageNet
base_model = ResNet50(include_top=False, 
                            classes=15,
                            input_shape=(1024,1024,3),
                            weights=None)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'images/train',
        # target_size=(1024, 1024),
        batch_size=128,
        class_mode='categorical')
model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        verbose=1)

2024-04-21 14:49:13.237228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 5859 images belonging to 15 classes.
Epoch 1/5
46/46 [==============================] - 1496s 32s/step - loss: 1.8988 - accuracy: 0.6706
Epoch 2/5
46/46 [==============================] - 1678s 37s/step - loss: 1.4587 - accuracy: 0.6894
Epoch 3/5
46/46 [==============================] - 1535s 33s/step - loss: 1.3889 - accuracy: 0.6902
Epoch 4/5
26/46 [===============>..............] - ETA: 11:01 - loss: 1.3562 - accuracy: 0.6869